Browse each detailed page, crawl all information of each house for sale

In [1]:
import pandas as pd
import numpy as np
import csv

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException

import undetected_chromedriver as uc
import time

chrome_options = Options()

Load data

In [2]:
# Load CSV file containing URLs to each completed house in the previous section
df = pd.read_csv(r"C:\Users\Admin\OneDrive - National Economics University\Documents\CRAWLER\URLs_data.csv") # "URLs_data.csv" 
# Convert Object thành String 
df['href'] = df['href'].astype(pd.StringDtype())

In [3]:
# Create a list variable containing urls to browse them one by one in the for loop
links = df['href'].to_list()

In [4]:
# Have an empty CSV output file, only the column names
with open('dataset.csv', 'w', newline='', encoding="utf-8") as csvfile:
    # writerow() method = write a single row into a CSV file.
    # writerows() method = write multiple rows to a CSV file.
    writerObj = csv.writer(csvfile)
    # for "nha dat ban"
    fields = ["Tiêu đề", "Địa chỉ","Ngày đăng", "Ngày hết hạn", "Loại tin", "Mã tin", "Người đăng", "Thông tin mô tả", "Diện tích", "Mức giá", "Mặt tiền","Đường vào","Hướng ban công","Hướng nhà","Số tầng", "Số phòng ngủ", "Số toilet", "Pháp lý", "Nội thất"]
    # # for "chung cu"
    # fields = ["Tiêu đề", "Dự án", "Khu vực","Ngày đăng", "Ngày hết hạn", "Loại tin", "Mã tin", "Người đăng", "Thông tin mô tả", "Diện tích", "Mức giá", "Hướng nhà", "Hướng ban công", "Số phòng ngủ", "Số toilet", "Pháp lý", "Nội thất"]
    writerObj.writerow(fields)

Computational and working functions

In [5]:
# Write a Single Row to a CSV File from a Python Dictionary
def add_single_row(info):
    with open(r"C:\Users\Admin\OneDrive - National Economics University\Documents\CRAWLER\dataset.csv", 'a+', newline='', encoding="utf-8") as csv_file:
        # for "nha dat ban"
        headers = ["Tiêu đề", "Địa chỉ","Ngày đăng", "Ngày hết hạn", "Loại tin", "Mã tin", "Người đăng", "Thông tin mô tả", "Diện tích", "Mức giá", "Mặt tiền","Đường vào","Hướng ban công","Hướng nhà","Số tầng", "Số phòng ngủ", "Số toilet", "Pháp lý", "Nội thất"]
        # # for "chung cu"
        # headers = ["Tiêu đề", "Dự án", "Khu vực","Ngày đăng", "Ngày hết hạn", "Loại tin", "Mã tin", "Người đăng", "Thông tin mô tả", "Diện tích", "Mức giá", "Hướng nhà", "Hướng ban công", "Số phòng ngủ", "Số toilet", "Pháp lý", "Nội thất"]
        writerObj = csv.DictWriter(csv_file, fieldnames=headers, delimiter=',')
        writerObj.writerow(info)

In [6]:
# Enter 2 List variables, return 1 Dictionary variable
def get_dict_from_2lists(title, value):
    dict_var = dict(zip(title, value)) # Convert 2 list into 1 dictionary
    #print("Important Fields of this House: " + str(dict_var))
    return dict_var

# Enter 2 Dictionary variables, update the second Dictionary to the first Dictionary variable, delete the second Dictionary
def merge(dict1, dict2):
    return {**dict1, **dict2} # (dict1.update(dict2))

# Gather all Fields into one dictionary
# for "nha dat ban"
def unite_into_1dict(tieude, diachi, nguoidang, thongtinmota, dict_temp):
    dict_final = {"Tiêu đề":tieude, 
                    "Địa chỉ": diachi,
                    "Người đăng": nguoidang,
                    "Thông tin mô tả": thongtinmota,
                    }
    return merge(dict_final, dict_temp)
# # for "chung cu"
# def unite_into_1dict(tieude, duan, khuvuc, nguoidang, thongtinmota, dict_temp):
#     dict_final = {"Tiêu đề":tieude, 
#                     "Dự án":duan, 
#                     "Khu vực": khuvuc,  
#                     "Người đăng": nguoidang,
#                     "Thông tin mô tả": thongtinmota                    
#                     }
#     return merge(dict_final, dict_temp)

In [7]:
# Run undetected chromedriver to get the data contained in the injected url link
def crawl_info(driver, link):
    driver.get(link)
    time.sleep(2) 
    
    # Title
    tieude = driver.find_element(By.CSS_SELECTOR, 'h1.re__pr-title.pr-title.js__pr-title').text
    
    # Prj + Addr
    # for "nha dat ban"
    addr = driver.find_element(By.CSS_SELECTOR, 'span.re__pr-short-description.js__pr-address').text
    diachi = addr.strip()
    # # for "chung cu"
    # addr = driver.find_element(By.CSS_SELECTOR, 'span.re__pr-short-description.js__pr-address').text
    # comma_index = addr.find(',')
    # duan = addr[:comma_index].strip()
    # khuvuc = addr[comma_index + 1:].strip()

    # Day + tin
    item_title_raw = driver.find_elements(By.CSS_SELECTOR, 'div.re__pr-short-info-item.js__pr-config-item span.title')
    item_value_raw = driver.find_elements(By.CSS_SELECTOR, 'div.re__pr-short-info-item.js__pr-config-item span.value')
    title_list = [] 
    value_list = []
    for i in range(0, len(item_title_raw)):
        title_list.append(item_title_raw[i].text)
        value_list.append(item_value_raw[i].text)
    dict1 = get_dict_from_2lists(title_list, value_list)

    # Features
    item_title_raw = driver.find_elements(By.CSS_SELECTOR, 'div.re__pr-specs-content-item span.re__pr-specs-content-item-title')
    item_value_raw = driver.find_elements(By.CSS_SELECTOR, 'div.re__pr-specs-content-item span.re__pr-specs-content-item-value')
    title_list = [] 
    value_list = [] 
    for i in range(0, len(item_title_raw)):
        title_list.append(item_title_raw[i].text)
        value_list.append(item_value_raw[i].text)
    dict2 = get_dict_from_2lists(title_list, value_list)
    dict_temp = merge(dict1, dict2)
    # print(dict_temp)
    
    # Contact person
    nguoidang = driver.find_element(By.CSS_SELECTOR, 'span.re__contact-name.js_contact-name').get_attribute('title').strip()
    
    # Detail
    # thongtinmota = driver.find_element(By.CSS_SELECTOR, 'div.re__section-body.re__detail-content.js__section-body.js__pr-description.js__tracking').text
    thongtinmota = driver.find_element(By.CLASS_NAME, "re__section-body").text
    
    # Dict_final
    # for "nha dat ban"
    dict_final = unite_into_1dict(tieude, diachi, nguoidang, thongtinmota, dict_temp)
    # # for "chung cu"
    # dict_final = unite_into_1dict(tieude, duan, khuvuc, nguoidang, thongtinmota, dict_temp)
    print(dict_final)
    driver.close()
    return dict_final

Run crawl

In [8]:
# If you encounter a situation where a home sale site cannot crawl,
# then crawl the URL stored in this "error" variable instead.
# This helps ensure urls_list index is not disturbed,
# However, it also means creating duplicates.
error = links[2] # URL at number 3 on the list
# print("Variable type=", type(error)) # Note, the path should always be of type string.
# print(error)

In [14]:
# MAIN
# driver = uc.Chrome() # version_main=114
path = r"C:\Users\Admin\OneDrive - National Economics University\Documents\chromedriver-win64\chromedriver.exe"
ser = Service(path)

# Tip: Edit "urls_list[index_start:index_end+1]" to crawl small pieces, instead of doing it all at once.
# Try gradually to see if there are any errors, e.g. [:5], [:100] and then up to several thousand data samples. To avoid fixing over and over again
for link in links[7500:9000]:
    try:
        # For each url of the detailed home sale page, we will crawl the corresponding information
        driver = uc.Chrome(service=ser)
        driver.maximize_window() 
        dict_final = crawl_info(driver, link) # Synthesize them into a Dictionary variable
        add_single_row(dict_final) # Then, write them down as a line in the dataset.csv file
        driver.quit()
#         time.sleep(2)
    except:
         # If something goes wrong, crawl the prepared url instead
#         i+=1
        driver = uc.Chrome(service=ser)  # or your preferred WebDriver
        dict_final = crawl_info(driver, error)
        add_single_row(dict_final)
    
driver.quit()

{'Tiêu đề': 'Bán nhà phố Chùa Láng, Đống Đa, ngõ kinh doanh, ô tô tránh, 2 thoáng. 74m2, 4 tầng, 14.7 tỷ', 'Địa chỉ': 'Phố Chùa Láng, Phường Láng Thượng, Đống Đa, Hà Nội', 'Người đăng': 'Đỗ Tuyết Nga', 'Thông tin mô tả': '- Nhà bán nằm tại vị trí đắc địa, ngõ ô tô tránh, khu phân lô, gần phố, giao thông thuận tiện, khu vực hạ tầng cao, gần nhiều trường đại học, kinh doanh sầm uất. Giao với các tuyến phố như Nguyễn Chí Thanh, Huỳnh Thúc Kháng, Pháo Đài Láng, Đường Láng,...\n\n- Nhà thiết kế 4 tầng thoáng trước sau, có ô chờ thang máy, hiện đang cho thuê kinh doanh.\n\n- Phía trong là khu nội trú dành cho sinh viên rất tiện lợi cho kinh doanh đa dạng mặt hàng.\n\n- Sổ chính chủ, vuông vắn.\n\n- Giá cực hợp lý, có thương lượng.\n\n- Lh:\n0963 899 ***\n- Miễn MG và TG.', 'Ngày đăng': '20/02/2024', 'Ngày hết hạn': '06/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '39074179', 'Diện tích': '74 m²', 'Mức giá': '14,7 tỷ', 'Số tầng': '4 tầng', 'Pháp lý': 'Sổ đỏ/ Sổ hồng', 'Nội thất': 'Đầy đủ'}

{'Tiêu đề': 'Bán tòa nhà ô tô tránh, sát mặt phố Hoàng Quốc Việt 93m2 9 tầng MT 7.2m', 'Địa chỉ': 'Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Cầu Giấy, Hà Nội', 'Người đăng': 'Mai Thị Xuân', 'Thông tin mô tả': 'Siêu phẩm phân lô, ô tô tránh, sát mặt phố Hoàng Quốc Việt với 9 tầng thang máy xịn xò.\nTòa nhà văn phòng, ở và cho thuê đều hái ra tiền.\nVới diện tích hơn 90m² mặt tiền hơn 7m.\nGiá chỉ nhỉnh 31tỷ.\nSổ đỏ pháp lý chuẩn, sẵn sàng giao dịch.\nLiên hệ ngay em Xuân chuyên nhà đất để được tư vấn và đi xem nhà hoàn toàn miễn phí.\nLh em Xuân\n0983 298 ***', 'Ngày đăng': '04/03/2024', 'Ngày hết hạn': '14/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '38508492', 'Diện tích': '95 m²', 'Mức giá': '31 tỷ', 'Mặt tiền': '7,2 m', 'Số tầng': '9 tầng'}
{'Tiêu đề': 'Máy in tiền Mỹ Đình, 8 tầng TM, 20m ra oto, 38 phòng, Đủ PCCC, DT 140m2 chỉ 1x tỷ', 'Địa chỉ': 'Đường Lê Quang Đạo, Phường Phú Đô, Nam Từ Liêm, Hà Nội', 'Người đăng': 'Mr Thiện', 'Thông tin mô tả': 'CCMN 8 tầng thang máy - ôtô cách 2

{'Tiêu đề': 'Bán biệt thự mini Yên Hòa - 74m2 - 10m ra mặt phố - nhà dân xây tâm huyết - ở cực sướng', 'Địa chỉ': 'Phường Yên Hòa, Cầu Giấy, Hà Nội', 'Người đăng': 'Đỗ Việt Anh', 'Thông tin mô tả': 'Bán biệt thự mini Yên Hòa - 74m², 4T, MT 8m - lô góc - 10m ra mặt phố - nhà dân xây tâm huyết - ở cực sướng - nhà siêu đẹp - nội thất đẹp - ô tô nhỏ qua - dân xây - ngõ thông.\n- Nhà 2 mặt thoáng vĩnh viễn bên cạnh là ngõ đi.\n- Nhà xây kiểu biệt thự 4 tầng có sân.\n- Ngõ rộng thông thoáng, ngõ đẹp nhất phố Yên Hoà.\n- Diện tích sàn xây dựng 270m², mỗi sàn 65m².\n4 phòng ngủ + khách bếp.\n- Sàn gỗ giáng hương, cửa và cầu thang gỗ Lim Lào.\n- Nội thất xịn, tất cả đều Tây Ban Nha từ bếp, chậu rửa đến hút mùi.\n- Tủ lạnh Side by side Electrolux. Điều hòa Đaikin các phòng.\n- Nhà có sân vườn riêng rất thanh cảnh.\nSổ đỏ cất két.', 'Ngày đăng': '28/02/2024', 'Ngày hết hạn': '14/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '39179433', 'Diện tích': '74 m²', 'Mức giá': '10,7 tỷ', 'Mặt tiền': '8 m'

{'Tiêu đề': 'Hiếm! PL Hàm Nghi! Dòng tiền gần 1tỷ/năm. Ô tô vào nhà! Gần 20PKK. Full NT. 75m2*7T. Nhỉnh 12 tỷ', 'Địa chỉ': 'Đường Trịnh Văn Bô, Phường Phương Canh, Nam Từ Liêm, Hà Nội', 'Người đăng': 'Nguyễn Xuan Việt', 'Thông tin mô tả': 'Hàng hiếm! Phân lô Hàm Nghi, Trịnh Văn Bô, Trung Tâm Mỹ Đình.\n+ DT 75m²*7tầng thang máy, 2 mặt ngõ, 16 khép kín. Full nội thất xịn. Dòng tiền đỉnh ~ 1 tỷ/năm.\n+ Vị trí đắc địa, ngõ thông, ô.\nGara ô tô. Khu Phân Lô, tiềm năng tăng giá cao.\n+ Đã kín phòng cho thuê. Khách chỉ việc khai thác dòng tiền.\n+ Nội thất chung: Khóa từ vân tay, camera từng tầng, điện thông minh. Không gian xanh, sạch, đẹp thiết bị đẳng cấp.\n+ SĐCC. Giá: 12.35 tỷ. LH:\n0988 906 ***', 'Ngày đăng': '10/03/2024', 'Ngày hết hạn': '20/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '37966262', 'Diện tích': '75 m²', 'Mức giá': '12,35 tỷ', 'Số tầng': '7 tầng'}
{'Tiêu đề': 'Phân lô Hoàng Cầu - vip - thông tứ tung - một bước tới hồ hoàng cầu - 62m2 x 7T thang máy, MT 5m', 'Địa chỉ': '

{'Tiêu đề': 'Tòa CCMN (70m2*6T*13PKK) đối diện hồ Mỗ Lao ngay Trần Phú - Big C Hồ Gươm PLaza Hà Đông HN', 'Địa chỉ': 'Đường Trần Phú, Phường Mỗ Lao, Hà Đông, Hà Nội', 'Người đăng': 'Minh Ngọc', 'Thông tin mô tả': 'Bán gấp siêu phẩm! Tòa CCMN (70m²*6T, 13PKK) full đồ nội thất. Ngay Big C Hồ Gươm PLaza Trần Phú - đối diện Hồ KĐT Mỗ Lao, Làng Việt Kiều Châu Âu Hà Đông HN. /giá chỉ nhỉnh 8,9 tỷ.\n* Diện tích sổ đỏ 70m², mặt tiền 4,5m mặt ngõ thông cực đẹp.\n- Nhà xây mới 5 tầng mỗi tầng 3PKK cực rộng đều có gách lửng bằng bê tông - diện tích mỗi phòng khoảng 25m, tất cả các phòng đều có 2 mặt thoáng nhiều ánh sáng tự nhiên, đầy đủ nội thất (giường, tủ, điều hòa nóng lạnh,... ). Tổng 13 phòng đã cho thuê kín giá từ 4tr đến 4,5 triệu/tháng.\n- Thu nhập khủng lên tới 60 triệu/tháng trở lên. (Cao hơn nhiều với lãi suất ngân hàng) Một kênh đầu tư sinh lời rất cao mà không bị mất giá của đồng tiền. (Full đồ nội thất như hình).\n- Hồ sơ xây dựng và bản thiết kế đầy đủ... Có lắp đặt hệ thống phòng

...

{'Tiêu đề': 'Bán nhà Trần Quang Diệu, ô tô tránh 7T thang máy, kinh doanh 85m2, 25 tỷ', 'Địa chỉ': 'Đường Trần Quang Diệu, Phường Ô Chợ Dừa, Đống Đa, Hà Nội', 'Người đăng': 'Dương Hoàng Hải', 'Thông tin mô tả': 'Bán nhà Trần Quang Diệu, ô tô tránh 7T thang máy, kinh doanh 85m², 25 tỷ.\nVị trí vip phố Trần Quang Diệu thông Thái Hà, Hoàng Cầu...\nThiết kế 7 tầng thang máy thông sàn phù hợp kinh doanh nhiều loại hình văn phòng, coffee, thời trang...\nSổ đẹp vuông vắn, giá thương lượng sâu.\nLiên hệ em Hải\n0982 896 ***', 'Ngày đăng': '29/02/2024', 'Ngày hết hạn': '15/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '38605034', 'Diện tích': '85 m²', 'Mức giá': '25 tỷ', 'Mặt tiền': '5 m', 'Đường vào': '8 m', 'Số tầng': '7 tầng', 'Số phòng ngủ': '5 phòng', 'Số toilet': '5 phòng', 'Pháp lý': 'Sổ đỏ/ Sổ hồng', 'Nội thất': 'Đầy đủ'}
{'Tiêu đề': 'Chỉ 3 tỷ 350 Phương Canh - nhà xây mới 5 tầng - MT 3,7m - lô góc siêu thoáng - ngõ rộng gần ô tô', 'Địa chỉ': 'đường Phương Canh, Phường Phương Canh, Na

{'Tiêu đề': 'Chính chủ bán nhà phân lô Trương Định - Hoàng Mai. 45m2*5T mặt tiền 4,5m. Ở kết hợp làm VP cực đỉnh', 'Địa chỉ': 'Đường Trương Định, Phường Tân Mai, Hoàng Mai, Hà Nội', 'Người đăng': 'Phan Hữu Cường', 'Thông tin mô tả': 'Ông anh em có nhà phân lô khu Tân Mai - Trương Định cần bán.\nỞ kết hợp làm VP cực đỉnh.\nThông số 45m²*5 tầng, mặt tiền 4,5m.\nXây dựng móng bè kiên cố - đẹp nhất khu phân lô.\nThiết kế: Gara + cầu thang giữa - tổng có 5 ngủ - sân thượng - đủ công năng.\nSổ phân lô đẹp, sẵn sàng giao dịch.\nE Cường\n0982 580 ***', 'Ngày đăng': '02/03/2024', 'Ngày hết hạn': '12/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '39212190', 'Diện tích': '45 m²', 'Mức giá': '8,6 tỷ', 'Mặt tiền': '4,5 m', 'Đường vào': '10 m', 'Số tầng': '5 tầng', 'Số phòng ngủ': '5 phòng', 'Số toilet': '4 phòng', 'Pháp lý': 'Sổ đỏ/ Sổ hồng', 'Nội thất': 'Đầy đủ.'}
{'Tiêu đề': 'Bán BT khu nhà ở Quốc Hội Tố Hữu, view đẹp, DT 218m2 x 3,5 tầng. Giá 42,5 tỷ. LH: 0964 769 ***', 'Địa chỉ': 'Đường Tố Hữu,

{'Tiêu đề': 'Chính chủ bán 106m2, MT 10m, giá hơn 9 tỷ - ngõ ô tô tải, Giải Phóng, Hoàng Mai', 'Địa chỉ': 'Đường Giải Phóng, Phường Thịnh Liệt, Hoàng Mai, Hà Nội', 'Người đăng': 'Bùi Xuân Hải', 'Thông tin mô tả': '- Ngõ cực thoáng thông khắp ngả, rộng 7m.\n\n- Cách vài chục mét là ra đường Giải Phóng, sát quận ủy Hoàng Mai.\n\n- Rất thích hợp xây chia lô hoặc xây làm VP cho thuê.\n\n- Tương lai rất gần đường có quy hoạch mở rộng ra đường Giải Phóng.\n\n- Sổ đỏ chính chủ, sẵn sàng giao dịch.', 'Ngày đăng': '28/02/2024', 'Ngày hết hạn': '09/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '38942056', 'Diện tích': '106 m²', 'Mức giá': '9,6 tỷ', 'Mặt tiền': '10 m', 'Đường vào': '7 m', 'Hướng ban công': 'Đông - Nam', 'Pháp lý': 'Sổ đỏ/ Sổ hồng.'}
{'Tiêu đề': 'Hạ chào bán gấp nhà Doãn Kế Thiện. 65m2, 5T, ngõ nông, KD + ở. Sổ đỏ CC. 9.6 tỷ còn TL. 0974 687 ***', 'Địa chỉ': 'Ngõ 105, Phố Doãn Kế Thiện, Phường Mai Dịch, Cầu Giấy, Hà Nội', 'Người đăng': 'Phạm Thúy Hoa', 'Thông tin mô tả': 'Hạ chào.

{'Tiêu đề': 'Căn hộ tầng 1, nhà M3 ngõ 97 Thái Thịnh, Đống Đa, Hà Nội', 'Địa chỉ': 'Nhà M3 ngõ 97, Phố Thái Thịnh, Phường Trung Liệt, Đống Đa, Hà Nội', 'Người đăng': 'Cảnh', 'Thông tin mô tả': 'Căn tầng 1 - DT sử dụng lên đến 52m².\n+ Bao gồm:\nPK.\n2 Pn.\nBếp, vsinh.\n\nVì ở tầng 1 nên có thẻ mơ kinh doanh nhỏ.\nLàm tóc, nail, phun săm.\nTạp hóa nhỏ lẻ..\n\nSổ đỏ: 19m².\nThực tế: 52m².\n\nCam ơn! 98.', 'Ngày đăng': '27/02/2024', 'Ngày hết hạn': '13/03/2024', 'Loại tin': 'Tin thường', 'Mã tin': '39175912', 'Diện tích': '19 m²', 'Mức giá': '2,58 tỷ', 'Số phòng ngủ': '2 phòng', 'Số toilet': '1 phòng'}
{'Tiêu đề': 'Gấp gấp chính chủ cần bán nhà 48m2 2 tầng tại Nguyễn Văn Cừ Long Biên Hà Nội, chỉ 3,8 tỷ gần TTam', 'Địa chỉ': 'Đường Nguyễn Văn Cừ, Phường Thạch Bàn, Long Biên, Hà Nội', 'Người đăng': 'Tài Bộ Đội', 'Thông tin mô tả': 'Gấp gấp: Chính chủ nhờ bán nhà 2 tầng 48m² hiện đg cho thuê 7tr/tháng, tại Nguyễn Văn Cừ, Long Biên, Hà Nội.\n\n- Đường trước nhà 3m ô tô đỗ cửa.\n- Tiện ích vô

{'Tiêu đề': 'Bán nhà Nguyễn Trãi - ô tô - dân xây - tặng full nội thất - 53m2, MT 4.5m - nhỉnh 7 tỷ', 'Địa chỉ': 'Đường Nguyễn Trãi, Phường Nhân Chính, Thanh Xuân, Hà Nội', 'Người đăng': 'Nguyễn Thị Thu Thanh', 'Thông tin mô tả': '- Nhà nằm vị trí giáp danh 3 quận: Đống Đa, Thanh Xuân, Cầu Giấy.\n- Ngõ ô tô đỗ cửa. Thông tứ tung.\n- Dân trí cao, an ninh siêu tốt.\n- Nhà dân xây kiên cố chắc chắn, thiết kế sang trọng hiện đại, chủ tận tâm tỉ mỉ chỉn chu sạch sẽ.\n- Nhà 2 thoáng vĩnh viễn, phòng nào cũng có cửa sổ thoáng sáng.\n- Giao thông thuận tiện di chuyển các tuyến phố lớn; Nguyễn Trãi, Nhân Hòa, Vũ Trọng Phụng, Quan Ngân.\n\nThiết Kế: DT 53 m² x 5.5 tầng, mt 4,5m, nhỉnh 7 tỷ.\n- Tầng 1: Khách + bếp + vs + Sân để xe rộng 5 - 6 chiếc ko phải đưa xe vào nhà.\n- Tầng 2,3,4: Mỗi tầng 2 phòng, vệ sinh giữ. (tổng 6 ngủ).\n- Tầng 5: Phòng thờ + Sân phơi.\n- Tầng 6: Mái che + sân vườn.\nPháp lý chuẩn, sẵn giao dịch.\nCô Nga muốn bán để chia tài sản cho các con.\nLH: Nguyễn Thanh (để liên h

In [10]:
# data = pd.read_csv('Dataset.csv')
# data

In [11]:
# data.info()

In [12]:
# data[data.duplicated()]